In [1]:
!pip install torch torchvision matplotlib numpy


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set up transformations: normalizing and converting to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the training and test sets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# Define classes in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:05<00:00, 28969794.60it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define a simple CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [4]:
def train_network(net, trainloader, criterion, optimizer, epochs=1):
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')

# Call the training function
train_network(net, trainloader, criterion, optimizer, epochs=1)


Epoch 1, Batch 2000: Loss: 2.286
Epoch 1, Batch 4000: Loss: 2.008
Epoch 1, Batch 6000: Loss: 1.758
Epoch 1, Batch 8000: Loss: 1.617
Epoch 1, Batch 10000: Loss: 1.549
Epoch 1, Batch 12000: Loss: 1.496
Finished Training


In [5]:
def evaluate_model(net, dataloader, criterion, device='cpu'):
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = test_loss / len(dataloader)
    print(f'Accuracy of the network on the test images: {accuracy:.2f}%')
    print(f'Average loss on the test images: {avg_loss:.4f}')
    return accuracy, avg_loss

# Evaluate the model without any attacks
accuracy, avg_loss = evaluate_model(net, testloader, criterion)


Accuracy of the network on the test images: 46.30%
Average loss on the test images: 1.4625


In [6]:
def noise_attack(image, epsilon):
    """Apply Gaussian noise attack to the input image."""
    # Adding noise scaled by epsilon
    noise = epsilon * torch.randn_like(image)
    # Adding the noise to the original image 
    perturbed_image = image + noise
    perturbed_image = torch.clamp(perturbed_image, 0, 1)  
    return perturbed_image


In [7]:
def evaluate_attack_performance(net, dataloader, criterion, attack, epsilons, device='cpu'):
    """Evaluate the model under different noise levels."""
    net.to(device)
    net.eval()
    performances = []

    for epsilon in epsilons:
        correct = 0
        total = 0
        test_loss = 0

        with torch.no_grad():
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                # Apply the attack
                perturbed_images = attack(images, epsilon)
                outputs = net(perturbed_images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        avg_loss = test_loss / len(dataloader)
        performances.append((epsilon, accuracy, avg_loss))
        print(f'For epsilon {epsilon}: Accuracy = {accuracy:.2f}%, Average Loss = {avg_loss:.4f}')

    return performances


In [9]:

# Values of epsilon for noise magnitude
epsilons = [0.01, 0.05, 1]

# Evaluate the model under noise attacks
performances = evaluate_attack_performance(net, testloader, criterion, noise_attack, epsilons)


For epsilon 0.01: Accuracy = 29.74%, Average Loss = 1.9614
For epsilon 0.05: Accuracy = 29.66%, Average Loss = 1.9623
For epsilon 1: Accuracy = 19.87%, Average Loss = 2.2939


In [11]:
import torch

# Set up device: Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cpu


In [12]:
def adversarial_training(net, trainloader, criterion, optimizer, epochs=1, epsilon=0.05):
    """Train the network on clean and adversarial examples."""
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass on clean images
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Creating adversarial examples using the noise attack
            perturbed_inputs = noise_attack(inputs, epsilon)

            # Forward pass on adversarial examples
            optimizer.zero_grad()
            outputs = net(perturbed_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1}: Loss: {running_loss / len(trainloader):.3f}')

    print('Finished Adversarial Training')

# Assuming net, trainloader, criterion, optimizer are already defined
adversarial_training(net, trainloader, criterion, optimizer, epochs=1, epsilon=0.05)


Epoch 1: Loss: 1.561
Finished Adversarial Training


In [15]:
def test_model(net, testloader, attack, epsilon):
    """Evaluate the model's accuracy under attack."""
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            if attack:
                images = attack(images, epsilon)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy with {attack.__name__ if attack else "no"} attack at epsilon={epsilon}: {accuracy:.2f}%')
    return accuracy


net_natural = Net()  # Assuming a new or reset instance of Net
train_network(net_natural, trainloader, criterion, optimizer, epochs=1)  # Train naturally
naturally_trained_accuracy = test_model(net_natural, testloader, noise_attack, 0.05)

# Use the adversarially trained model
adversarially_trained_accuracy = test_model(net, testloader, noise_attack, 0.05)

# Output the results
print(f'Accuracy of naturally trained model against attack: {naturally_trained_accuracy:.2f}%')
print(f'Accuracy of adversarially trained model against noise attack: {adversarially_trained_accuracy:.2f}%')


Epoch 1, Batch 2000: Loss: 2.305
Epoch 1, Batch 4000: Loss: 2.305
Epoch 1, Batch 6000: Loss: 2.306
Epoch 1, Batch 8000: Loss: 2.304
Epoch 1, Batch 10000: Loss: 2.305
Epoch 1, Batch 12000: Loss: 2.305
Finished Training
Accuracy with noise_attack attack at epsilon=0.05: 11.18%
Accuracy with noise_attack attack at epsilon=0.05: 47.14%
Accuracy of naturally trained model against attack: 11.18%
Accuracy of adversarially trained model against noise attack: 47.14%


In [29]:
def test_cw_attack_iterations(net, testloader, device, iterations_list):
    net.to(device)
    net.eval()
    results = []
    
    for max_iter in iterations_list:
        correct = 0
        total = 0
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            perturbed_images = cw_attack(net, images, labels, device, max_iter=max_iter)
            
            outputs = net(perturbed_images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Accuracy under C-W attack with {max_iter} iterations: {accuracy:.2f}%')
        results.append((max_iter, accuracy))
    
    return results

# Example: Test with varying iterations
iteration_counts = [3, 7, 12]
test_cw_attack_iterations(net, testloader, device, iteration_counts)



Accuracy under C-W attack with 3 iterations: 19.96%
Accuracy under C-W attack with 7 iterations: 6.41%
Accuracy under C-W attack with 12 iterations: 1.59%


[(3, 19.96), (7, 6.41), (12, 1.59)]

In [90]:
import torch

def cw_attack(model, images, labels, device, max_iter=1000, c=1e-4):
    
    model.eval()
    
    # Get original images and labels on the correct device
    images = images.to(device)
    labels = labels.to(device)

    
    perturbation = torch.zeros_like(images, requires_grad=True).to(device)
    
    # Set up optimizer for the perturbation
    optimizer = torch.optim.Adam([perturbation], lr=0.01)
    
    # Define the target class as the least likely predicted class by the model (untargeted attack)
    target_labels = labels

    for i in range(max_iter):
        optimizer.zero_grad()
        
        # Add perturbation to images and clip to ensure valid pixel range
        adv_images = images + perturbation
        adv_images = torch.clamp(adv_images, 0, 1)
        
        # Calculate loss
        outputs = model(adv_images)
        real = (outputs.gather(1, labels.unsqueeze(1)).squeeze(1))
        other = (outputs.max(1)[0] - 1000 * (labels == outputs.max(1)[1]).float())
        f = torch.clamp(other - real + 10, min=0)  # 10 is kappa (confidence)
        loss = torch.sum(perturbation ** 2) + c * torch.sum(f)

        # Backward pass
        loss.backward()
        optimizer.step()

    # Return the perturbed images
    return adv_images.detach()

# Example usage in your testing function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
iteration_counts = [3, 7, 20]
test_cw_attack_iterations(net, testloader, device, iteration_counts)


Accuracy under C-W attack with 3 iterations: 26.46%
Accuracy under C-W attack with 7 iterations: 24.16%
Accuracy under C-W attack with 20 iterations: 24.92%


[(3, 26.46), (7, 24.16), (20, 24.92)]

In [86]:
def adversarial_training(net, trainloader, criterion, optimizer, device, epochs=1):
    net.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Generate adversarial examples
            perturbed_inputs = cw_attack(net, inputs, labels, device, max_iter=10)
            
            # Forward pass with adversarial examples
            outputs = net(perturbed_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Print statistics
            running_loss += loss.item()
            if (i + 1) % 2000 == 0:  # print every 2000 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    print('Finished Adversarial Training')


In [87]:
def evaluate_model_under_attack(net, testloader, device, iterations_list):
    net.to(device)
    net.eval()
    results = []
    
    for max_iter in iterations_list:
        correct = 0
        total = 0
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            perturbed_images = cw_attack(net, images, labels, device, max_iter=max_iter)
            
            outputs = net(perturbed_images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        results.append((max_iter, accuracy))
    
    return results


In [89]:
# Assume 'net' is your model, and 'net_adv' is a new instance of the same model for adversarial training
net_adv = Net()  # Re-instantiate if needed
net_adv.to(device)
optimizer_adv = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)

# Natural training (already provided above)
# Adversarial training
adversarial_training(net_adv, trainloader, criterion, optimizer_adv, device, epochs=1)

# Evaluate both models
iterations_list = [3]  # Adjust as needed for more thorough testing
accuracy_natural = evaluate_model_under_attack(net, testloader, device, iterations_list)
accuracy_adv_trained = evaluate_model_under_attack(net_adv, testloader, device, iterations_list)

print(f'Accuracy of naturally trained model against C-W attack: {accuracy_natural[0][1]:.2f}%')
print(f'Accuracy of adversarially trained model against C-W attack: {accuracy_adv_trained[0][1]:.2f}%')


Epoch 1, Batch 2000: Loss: 2.297
Epoch 1, Batch 4000: Loss: 2.086
Epoch 1, Batch 6000: Loss: 1.858
Epoch 1, Batch 8000: Loss: 1.736
Epoch 1, Batch 10000: Loss: 1.666
Epoch 1, Batch 12000: Loss: 1.599
Finished Adversarial Training
Accuracy of naturally trained model against C-W attack: 26.46%
Accuracy of adversarially trained model against C-W attack: 43.38%


In [65]:
def cw_attack(model, image, label, device, kappa=0, c=0.1, lr=0.01, max_iter=100):
    image = image.detach().to(device)  # Ensure no gradients for the original image
    perturbation = torch.zeros_like(image, requires_grad=True, device=device)  # Ensure gradients

    optimizer = torch.optim.Adam([perturbation], lr=lr)

    for _ in range(max_iter):
        optimizer.zero_grad()
        perturbed_image = apply_perturbation_and_preserve_gradients(image, perturbation)
        loss = compute_loss(model, perturbed_image, label, kappa, c, perturbation)
        loss.backward()
        optimizer.step()

    final_perturbed_image = apply_perturbation_and_preserve_gradients(image, perturbation)
    return final_perturbed_image



In [68]:
import torch

def initialize_perturbation(image, device):
    image = image.detach().to(device)
    perturbation = torch.zeros_like(image, device=device, requires_grad=True)
    return image, perturbation

def apply_perturbation_and_preserve_gradients(image, perturbation):
    perturbed_image = image + perturbation
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    perturbed_image.requires_grad_(True)  # Ensure gradients are preserved
    return perturbed_image

def compute_loss(model, perturbed_image, label, kappa, c, perturbation):
    model.train()  # Enable training mode for gradient computation
    outputs = model(perturbed_image)
    real_class_logits = outputs.gather(1, label.unsqueeze(1)).squeeze(1)
    max_other_logits = (outputs - 1e4 * torch.eye(10, device=perturbed_image.device)[label]).max(1)[0]
    cw_loss = torch.clamp(real_class_logits - max_other_logits + kappa, min=0).mean()
    l2_loss = c * torch.norm(perturbation)
    total_loss = cw_loss + l2_loss
    model.eval()  # Switch back to evaluation mode
    return total_loss

def cw_attack(model, image, label, device, kappa=0, c=0.1, lr=0.01, max_iter=100):
    image, perturbation = initialize_perturbation(image, device)
    optimizer = torch.optim.Adam([perturbation], lr=lr)

    for _ in range(max_iter):
        optimizer.zero_grad()
        perturbed_image = apply_perturbation_and_preserve_gradients(image, perturbation)
        loss = compute_loss(model, perturbed_image, label, kappa, c, perturbation)
        loss.backward()
        optimizer.step()

    final_perturbed_image = apply_perturbation_and_preserve_gradients(image, perturbation)
    return final_perturbed_image
